In [1]:
!pip install opensearch-py

     |████████████████████████████████| 266kB 22.7MB/s eta 0:00:01
     |████████████████████████████████| 163kB 85.5MB/s eta 0:00:01
     |████████████████████████████████| 71kB 24.6MB/s eta 0:00:01
     |████████████████████████████████| 153kB 59.4MB/s eta 0:00:01
     |████████████████████████████████| 235kB 74.8MB/s eta 0:00:01
     |████████████████████████████████| 143kB 68.9MB/s eta 0:00:01
     |████████████████████████████████| 71kB 23.8MB/s eta 0:00:01
ERROR: openapi-schema-validator 0.6.2 has requirement jsonschema<5.0.0,>=4.19.1, but you'll have jsonschema 4.19.0 which is incompatible.
ERROR: mosaic-ai-client 1.0.0 has requirement matplotlib==3.1.1, but you'll have matplotlib 3.6.0 which is incompatible.
ERROR: jsonschema-path 0.3.2 has requirement referencing<0.32.0,>=0.28.0, but you'll have referencing 0.33.0 which is incompatible.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
from opensearchpy import OpenSearch
host = 'vpc-aw-fdc-opensearch-qa-01-klbmt4ru7oksvsemnnmjpohfw4.us-east-1.es.amazonaws.com'
port = 443
auth = ('qa-developer', 'wF5341W8=~') # For testing only. Don't store credentials in code.
#ca_certs_path = '/full/path/to/root-ca.pem' # Provide a CA bundle if you use intermediate CAs with your root CA.

# Create the client with SSL/TLS enabled, but hostname verification disabled.
client = OpenSearch(
    hosts = [{'host': host, 'port': port}],
    http_compress = True, # enables gzip compression for request bodies
    http_auth = auth,
    use_ssl = True,
    verify_certs = True,
    ssl_assert_hostname = False,
    ssl_show_warn = False
)

In [67]:
query = {
  "size" : 10000,
  "sort" : [
        { "@timestamp" : {"order" : "desc"}}
  ],
  "query": {
    "bool": {
        "must": [
            {"match_phrase" : {"kubernetes.labels.app": "fosfor-gatekeeper"}},
            {"regexp": {"message": "[0-9]{1,3}.[0-9]{1,3}.[0-9]{1,3}.[0-9]{1,3}"}}
        ],
        "filter" : { "bool" :   {   
            "must_not": [
                {"regexp" : {"message": "elb"}},
                {"regexp" : {"message": "gatewayconfig"}},
            ]
            }
        }
      }
    }
}
    
response = client.search(
    body = query,
    index = 'filebeat-qa-*'
)

#response

In [74]:
import sys

l_arr = []
for hit in response['hits']['hits']:
    s_arr = []
    str = hit["_source"]["message"].split()
    if len(str) > 9:
        s_arr.append(str[6])
        s_arr.append(str[8])
        s_arr.append(str[9])
        l_arr.append(s_arr)

import pandas as pd
df = pd.DataFrame(l_arr, columns=['URL', 'CODE', 'TIME'])

df["TIME"] = pd.to_numeric(df['TIME'], errors='coerce')
df.dropna(subset=['TIME'], inplace=True)
df.sort_values(by='TIME', inplace=True, ascending=False)

In [80]:
df[10:100].head(50)

,URL,CODE,TIME
724,/fdc/578.515f3538683840c3.bundle.js,200,6971317.0
3218,/fdc/578.515f3538683840c3.bundle.js,200,6971317.0
1373,/fdc/578.515f3538683840c3.bundle.js,200,6971317.0
3203,/fdc/578.515f3538683840c3.bundle.js,200,6971317.0
1913,/fdc/578.515f3538683840c3.bundle.js,200,6971317.0
9178,/fdc/578.515f3538683840c3.bundle.js,200,6971317.0
6647,/fdc/578.515f3538683840c3.bundle.js,200,6971317.0
2193,/fdc/578.515f3538683840c3.bundle.js,200,6971317.0
2738,/fdc/578.515f3538683840c3.bundle.js,200,6971317.0
5427,/fdc/578.515f3538683840c3.bundle.js,200,6971317.0


In [83]:
df2 = df[df["TIME"] > 60000]

In [82]:
df2

,URL,CODE,TIME
9438,/monitor-pipeline/vendors-node_modules_fosfor-...,200,35177432.0
1541,/monitor-pipeline/vendors-node_modules_fosfor-...,200,35177432.0
1785,/monitor-pipeline/vendors-node_modules_fosfor-...,200,35177432.0
2999,/monitor-pipeline/vendors-node_modules_fosfor-...,200,35177432.0
2947,/pipeline/vendors-node_modules_date-io_date-fn...,200,18603544.0
...,...,...,...
4073,/oauth/v2/logout,200,66.0
2955,/workspace/v1/story/664f2e29ff95990b858bc41c,200,66.0
354,/oauth/v2/logout,200,66.0
5267,/lenientry/v2/fetchAccountInfo,200,65.0
